In [ ]:
# --- 0. Instalación y Setup ---
!pip install roboflow ultralytics -q
from ultralytics import YOLO
import os
import yaml
from roboflow import Roboflow
from google.colab import drive
from IPython.display import clear_output, display
import pandas as pd
import logging

# Desactiva los logs detallados de Ultralytics durante la validación
logging.getLogger('ultralytics').setLevel(logging.ERROR)

# --- 1. CONFIGURACIÓN PRINCIPAL ---
ROBOFLOW_API_KEY = "tZR7XZhFvySrbVM3hfH4"
ROBOFLOW_WORKSPACE = "luchini"
ROBOFLOW_PROJECT = "datasetenproceso-irzcf"
VALIDATION_DATASET_VERSION = 15
BASE_TRAINING_PATH = '/content/drive/MyDrive/YOLOv8_Training'

SAFE_VALIDATION_BATCH = 16

print("Configuración cargada. Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Drive Montado.")

# --- 2. Descargar Dataset de VALIDACIÓN (Una sola vez) ---
print(f"\nDescargando dataset de validación (v{VALIDATION_DATASET_VERSION}) desde Roboflow...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(VALIDATION_DATASET_VERSION)
dataset = version.download("yolov8")

validation_data_yaml_path = f'{dataset.location}/data.yaml'

with open(validation_data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)
data_yaml['train'] = f"{dataset.location}/train/images"
data_yaml['val'] = f"{dataset.location}/valid/images"
if os.path.exists(f"{dataset.location}/test/images"):
    data_yaml['test'] = f"{dataset.location}/test/images"
with open(validation_data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

clear_output(wait=True)
print("✅ Drive Montado.")
print(f"✅ Dataset de Validación (v{VALIDATION_DATASET_VERSION}) descargado y listo.")

# --- 3. Bucle de Evaluación de Modelos ---

print(f"\nIniciando evaluación de todos los modelos en: {BASE_TRAINING_PATH}")
print(f"ADVERTENCIA: Esto puede tardar MUCHO tiempo (varios minutos por modelo).")

all_results = []

try:
    model_folders = [
        f for f in os.listdir(BASE_TRAINING_PATH)
        if os.path.isdir(os.path.join(BASE_TRAINING_PATH, f))
    ]
except FileNotFoundError:
    print(f"❌ ERROR: No se pudo encontrar la carpeta base: {BASE_TRAINING_PATH}")
    model_folders = []

if not model_folders:
    print("No se encontraron carpetas de modelos para evaluar.")
else:
    print(f"Se encontraron {len(model_folders)} posibles carpetas de modelos.")

for run_name in model_folders:
    run_path = os.path.join(BASE_TRAINING_PATH, run_name)
    model_path = os.path.join(run_path, 'weights/best.pt')
    args_path = os.path.join(run_path, 'args.yaml')

    print(f"\n--- Procesando: {run_name} ---")

    if not os.path.exists(model_path) or not os.path.exists(args_path):
        print("... ❌ Omitiendo. No se encontró 'best.pt' y/o 'args.yaml'.")
        continue

    try:
        # Cargar args para obtener imgsz y batch
        with open(args_path, 'r') as f:
            args = yaml.safe_load(f)

        val_imgsz = args.get('imgsz', 640)
        val_batch_from_args = args.get('batch', SAFE_VALIDATION_BATCH)

        # Si el batch guardado es -1 (o 0), usamos el default seguro (SAFE_VALIDATION_BATCH)
        if val_batch_from_args <= 0:
            val_batch = SAFE_VALIDATION_BATCH
            print(f"... 'batch' original era {val_batch_from_args}. Usando {val_batch} para validación.")
        else:
            val_batch = val_batch_from_args

        # Cargar modelo
        print(f"... Cargando 'best.pt' (imgsz: {val_imgsz}, batch: {val_batch})")
        model = YOLO(model_path)

        # Ejecutar validación (verbose=False para no inundar la salida)
        print("... Ejecutando validación...")
        metrics = model.val(
            data=validation_data_yaml_path,
            split='val',
            imgsz=val_imgsz,
            batch=val_batch,
            verbose=False
        )

        # Guardar las métricas principales
        all_results.append({
            'Modelo': run_name,
            'Mask mAP50-95': metrics.seg.map,
            'Mask mAP50': metrics.seg.map50,
            'Box mAP50-95': metrics.box.map,
            'imgsz': val_imgsz,
            'batch_train': val_batch_from_args # Guardamos el batch de entreno original
        })
        print(f"... ✅ Completado. Mask mAP50-95: {metrics.seg.map:.4f}")

    except Exception as e:
        print(f"... ❌ ERROR al procesar este modelo: {e}")

# --- 4. Mostrar Resultados Finales ---

print("\n\n--- 🏆 TABLA DE COMPARACIÓN FINAL 🏆 ---")

if not all_results:
    print("No se pudo validar ningún modelo.")
else:
    # Convertir la lista de resultados en un DataFrame de Pandas
    results_df = pd.DataFrame(all_results)

    # Ordenar el DataFrame por la métrica principal (Mask mAP50-95)
    results_df_sorted = results_df.sort_values(by='Mask mAP50-95', ascending=False)

    # Formatear los números para mejor legibilidad
    results_df_sorted['Mask mAP50-95'] = results_df_sorted['Mask mAP50-95'].map('{:.4f}'.format)
    results_df_sorted['Mask mAP50'] = results_df_sorted['Mask mAP50'].map('{:.4f}'.format)
    results_df_sorted['Box mAP50-95'] = results_df_sorted['Box mAP50-95'].map('{:.4f}'.format)

    # Mover la columna 'batch_train' para que sea más clara
    cols = ['Modelo', 'Mask mAP50-95', 'Mask mAP50', 'Box mAP50-95', 'imgsz', 'batch_train']

    # Asegurarnos de que todas las columnas existan antes de reordenar
    final_cols = [c for c in cols if c in results_df_sorted.columns]
    display(results_df_sorted[final_cols])

✅ Drive Montado.
✅ Dataset de Validación (v15) descargado y listo.

Iniciando evaluación de todos los modelos en: /content/drive/MyDrive/YOLOv8_Training
ADVERTENCIA: Esto puede tardar MUCHO tiempo (varios minutos por modelo).
Se encontraron 23 posibles carpetas de modelos.

--- Procesando: Coffee_Cherry_Segmentation_Run1 ---
... Cargando 'best.pt' (imgsz: 640, batch: 8)
... Ejecutando validación...
... ✅ Completado. Mask mAP50-95: 0.7822

--- Procesando: Coffee_Cherry_Seg_SmallModel_Run2 ---
... Cargando 'best.pt' (imgsz: 640, batch: 32)
... Ejecutando validación...
... ✅ Completado. Mask mAP50-95: 0.7857

--- Procesando: Coffee_Cherry_Seg_MediumModel_Run3 ---
... Cargando 'best.pt' (imgsz: 640, batch: 32)
... Ejecutando validación...
... ✅ Completado. Mask mAP50-95: 0.7935

--- Procesando: Coffee_Cherry_Seg_RectInput_Run4 ---
... Cargando 'best.pt' (imgsz: [640, 480], batch: 32)
... Ejecutando validación...
... ✅ Completado. Mask mAP50-95: 0.8137

--- Procesando: Coffee_Seg_Debug_Run 

,Modelo,Mask mAP50-95,Mask mAP50,Box mAP50-95,imgsz,batch_train
12,Coffee_Seg_L-Model_StableAug_1088px_Batch16_v1,0.8785,0.9176,0.8803,1088,8
14,Coffee_Seg_L-1280_SGDcos_AugsConservadores_v1,0.8567,0.8986,0.8606,1280,16
15,Coffee_Seg_L-1536_rect,0.8544,0.8894,0.8494,1536,-1
16,Coffee_Seg_L-1536_rect2,0.8544,0.8894,0.8494,1536,-1
17,Coffee_Seg_x-1536_rect,0.8532,0.8897,0.8474,1536,-1
18,Coffee_Seg_x-1536_rect2,0.8532,0.8897,0.8474,1536,-1
6,Coffee_Seg_L-Model_AggressiveAug_v1,0.8224,0.9143,0.8661,640,32
7,Coffee_Seg_L-Model_AggressiveAug_v1_350y8x,0.8184,0.9051,0.8621,640,32
10,Coffee_Seg_L-Model_AggressiveAug_Finetuned,0.8169,0.9103,0.8634,640,32
3,Coffee_Cherry_Seg_RectInput_Run4,0.8137,0.9014,0.8590,"[640, 480]",32
